<a href="https://colab.research.google.com/github/digitalcoya/digitalcoya.github.io/blob/main/fotoapp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image, ImageFilter, ImageOps
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files
import os



def cargar_imagen_colab():
    print("Suba una imagen…")
    uploaded = files.upload()

    if len(uploaded) == 0:
        raise FileNotFoundError("No se subió ninguna imagen.")

    nombre = next(iter(uploaded.keys()))
    print(f"Imagen cargada: {nombre}")
    return nombre




def redimensionar_imagen(ruta, plataforma):

    tamaños = {
        "youtube":  (1280, 720),
        "instagram": (1080, 1080),
        "twitter": (1200, 675),
        "facebook": (1200, 630)
    }

    plataforma = plataforma.lower()

    if plataforma not in tamaños:
        raise ValueError("Plataforma no válida.")

    new_width, new_height = tamaños[plataforma]

    img = Image.open(ruta)

    original_width, original_height = img.size
    ratio_original = original_width / original_height
    ratio_nuevo = new_width / new_height

    if ratio_original > ratio_nuevo:
        new_h = int(new_width / ratio_original)
        resized_img = img.resize((new_width, new_h), Image.Resampling.LANCZOS)
    else:
        new_w = int(new_height * ratio_original)
        resized_img = img.resize((new_w, new_height), Image.Resampling.LANCZOS)

    lienzo = Image.new("RGB", (new_width, new_height), color=(0, 0, 0))
    pos_x = (new_width - resized_img.width) // 2
    pos_y = (new_height - resized_img.height) // 2
    lienzo.paste(resized_img, (pos_x, pos_y))

    nombre = f"imagen_{plataforma}.jpg"
    lienzo.save(nombre)

    print(f"Imagen redimensionada guardada como: {nombre}")
    files.download(nombre)

    return nombre



def equalizar_contraste(ruta_imagen):

    img_original = cv2.imread(ruta_imagen)
    if img_original is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen.")

    img_ycrcb = cv2.cvtColor(img_original, cv2.COLOR_BGR2YCrCb)
    Y, Cr, Cb = cv2.split(img_ycrcb)
    Y_equalized = cv2.equalizeHist(Y)

    img_ecualizada = cv2.merge([Y_equalized, Cr, Cb])
    img_ecualizada = cv2.cvtColor(img_ecualizada, cv2.COLOR_YCrCb2BGR)

    img_original_rgb = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
    img_ecualizada_rgb = cv2.cvtColor(img_ecualizada, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 6))
    plt.subplot(1,2,1)
    plt.imshow(img_original_rgb)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.imshow(img_ecualizada_rgb)
    plt.title("Ecualizada")
    plt.axis("off")

    plt.show()

    salida = "contraste_resultado.jpg"
    cv2.imwrite(salida, img_ecualizada)

    print(f"Imagen guardada como: {salida}")
    files.download(salida)

    return salida




FILTROS = {
    "ORIGINAL": None,
    "BLUR": ImageFilter.BLUR,
    "CONTOUR": ImageFilter.CONTOUR,
    "DETAIL": ImageFilter.DETAIL,
    "EDGE ENHANCE": ImageFilter.EDGE_ENHANCE,
    "EDGE ENHANCE MORE": ImageFilter.EDGE_ENHANCE_MORE,
    "EMBOSS": ImageFilter.EMBOSS,
    "FIND EDGES": ImageFilter.FIND_EDGES,
    "SHARPEN": ImageFilter.SHARPEN,
    "SMOOTH": ImageFilter.SMOOTH
}

def aplicar_filtro(imagen_path, filtro):

    filtro = filtro.upper()
    if filtro not in FILTROS:
        raise ValueError("Filtro inválido.")

    img = Image.open(imagen_path)

    if FILTROS[filtro] is None:
        img_filtrada = img
    else:
        img_filtrada = img.filter(FILTROS[filtro])

    plt.imshow(img_filtrada)
    plt.title(f"Filtro: {filtro}")
    plt.axis("off")
    plt.show()

    salida = f"filtro_{filtro}.jpg"
    img_filtrada.save(salida)
    files.download(salida)

    return salida



def generar_boceto(imagen_path, modo="lapiz"):

    img = Image.open(imagen_path)
    gris = ImageOps.grayscale(img)
    suave = gris.filter(ImageFilter.GaussianBlur(radius=2))
    bordes = suave.filter(ImageFilter.FIND_EDGES)

    if modo == "lapiz":
        boceto = ImageOps.invert(bordes)
        boceto = ImageOps.autocontrast(boceto)
    else:
        arr = np.array(bordes)
        umbral = 90
        binar = (arr > umbral) * 255
        boceto = Image.fromarray(binar.astype("uint8"))

    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.axis("off")
    plt.title("Original")

    plt.subplot(1,2,2)
    plt.imshow(boceto, cmap="gray")
    plt.axis("off")
    plt.title("Boceto")

    plt.show()

    salida = f"boceto_{modo}.jpg"
    boceto.save(salida)
    files.download(salida)

    return salida



def menu():

    print("CARGAR IMAGEN: ")
    imagen = cargar_imagen_colab()

    while True:
        print("\n MENU FOTOAPP ")
        print("1) Redimensionar")
        print("2) Ajustar contraste")
        print("3) Aplicar filtro")
        print("4) Generar boceto")
        print("5) Salir")

        op = input("Opción: ")

        if op == "1":
            plataforma = input("Plataforma (youtube/instagram/twitter/facebook): ")
            redimensionar_imagen(imagen, plataforma)

        elif op == "2":
            equalizar_contraste(imagen)

        elif op == "3":
            print("Filtros:", list(FILTROS.keys()))
            filtro = input("Filtro: ")
            aplicar_filtro(imagen, filtro)

        elif op == "4":
            modo = input("Modo (lapiz/binarizado): ")
            generar_boceto(imagen, modo)

        elif op == "5":
            print("Saliendo...")
            break

        else:
            print("Opción inválida.")


# Para iniciar menú:
menu_colab()